In [24]:
import numpy as np
from pathlib import Path
from util import read_refdata, isotope
# for path in Path('.').iterdir():
# path = Path('L1.25.dat')
# if path.suffix != '.dat':
#     continue
path = Path('/home/jiangrz/ssd/GitHub/rproc/data/sne/bisterzo2010/30ST1.dat')
with open(path, 'r') as file:
    spltlines = file.read().splitlines()
mfrac, nfrac = read_refdata('/home/jiangrz/ssd/GitHub/rproc/data/ref/sol_asplund09.dat')
XH_sol = np.log10(nfrac)
XH_sol += (-XH_sol[0]+12)
with open('p.dat', 'w') as file:
    feh = -float(path.stem[:2])/10
    for line in spltlines:
        if line.startswith('#'):
            continue
        name, xfe = line.split()
        xfe = float(xfe)
        iso = isotope(name)
        xh_sol = XH_sol[iso.Z-1]
        xh = xfe + feh
        nfrac = np.power(10, xh+xh_sol-12)
        new_line = '%-5s%.6E\n'%(name, nfrac)
        file.write(new_line)
        # break

/tmp/ipykernel_2742061/2066082320.py:12: RuntimeWarning: divide by zero encountered in log10
  XH_sol = np.log10(nfrac)
